# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nhl_correlation():
    # YOUR CODE HERE
    # Data Cleaning: US Cities Dataset
    # Import Dataset
    cities = pd.read_html("Assets/wikipedia_data.html")[1].iloc[:-1, [0, 3, 5, 6, 7, 8]]
    # Data Cleaning Operations
    nhl_cities_df = cities.iloc[: ,[0, 1, -1]]
    nhl_cities_df['NHL'].replace('\[.*\]', '', inplace = True, regex = True)
    nhl_cities_df['NHL'].replace(['RangersIslandersDevils', 'KingsDucks', '—', ''],['Rangers', 'Kings', np.nan, np.nan], inplace = True) 
    nhl_cities_df['NHL'] = nhl_cities_df['NHL'].str.strip()
    nhl_cities_df = nhl_cities_df.dropna()
    nhl_cities_df.columns = ["city", "pop", "nhl"]
    nhl_cities_df['pop'] = pd.to_numeric(nhl_cities_df['pop'])
    multiple_team_cities = [
        {'city' : 'New York City', 'pop' : 0, 'nhl' : 'Islanders'},
        {'city' : 'New York City', 'pop' : 0, 'nhl' : 'Devils'},
        {'city' : 'Los Angeles', 'pop' : 0, 'nhl' : 'Ducks'}
    ]
    nhl_cities_df = pd.concat([nhl_cities_df, pd.DataFrame(multiple_team_cities)])
    nhl_cities = nhl_cities_df.drop_duplicates("city").sort_values(by = "city")
    # Extract Series containing data: "Population by Region"
    population_by_region = nhl_cities["pop"]
    # Data Cleaning: NHL Teams Dataset
    # Import Dataset
    nhl_df = pd.read_csv("Assets/nhl.csv")
    # Data Cleaning Operations
    nhl_df = nhl_df[nhl_df.year == 2018].iloc[:, [0, 2, 3]]
    nhl_df = nhl_df[~nhl_df.team.str.contains('Division')]
    nhl_df.team = nhl_df.team.replace('*', '')
    nhl_df["city"] = 0
    for i in range(len(nhl_cities_df)):
        nhl_df["city"][nhl_df.team.str.contains(nhl_cities_df.nhl.iloc[i])] = nhl_cities_df.city.iloc[i]
        nhl_df["team"][nhl_df.team.str.contains(nhl_cities_df.nhl.iloc[i])] = nhl_cities_df.nhl.iloc[i]
    nhl_df = nhl_df.astype({'W': int, 'L': int})
    total_df = nhl_df.groupby("city").sum()
    total_df["W/L"] = total_df["W"] / (total_df["W"] + total_df["L"])
    total_df = total_df.sort_values(by = "city")
    # Extract Series containing data: "W/L Ratio by Region"
    win_loss_by_region = total_df["W/L"]
    # Code Assessments
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    #raise NotImplementedError()
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [2]:
nhl_correlation()

/Users/juandiegoocampo/.pyenv/versions/3.9.5/lib/python3.9/site-packages/pandas/core/series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-1-af2efee55b0f>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nhl_cities_df['NHL'] = nhl_cities_df['NHL'].str.strip()
<ipython-input-1-af2efee55b0f>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

0.012308996455744264

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nba_correlation():
    # YOUR CODE HERE
    # Data Cleaning: US Cities Dataset
    # Import Dataset
    cities = pd.read_html("Assets/wikipedia_data.html")[1].iloc[:-1, [0, 3, 5, 6, 7, 8]]
    nba_cities_df = cities.iloc[: ,[0, 1, -2]]
    nba_cities_df['NBA'].replace('\[.*\]', '', inplace = True, regex = True)
    nba_cities_df['NBA'].replace(['KnicksNets', 'LakersClippers', '—', ''],['Knicks', 'Lakers', np.nan, np.nan], inplace = True) 
    nba_cities_df['NBA'] = nba_cities_df['NBA'].str.strip()
    nba_cities_df = nba_cities_df.dropna()
    nba_cities_df.columns = ["city", "pop", "nba"]
    nba_cities_df['pop'] = pd.to_numeric(nba_cities_df['pop'])
    multiple_team_cities = [
        {'city' : 'New York City', 'pop' : 0, 'nba' : 'Nets'},
        {'city' : 'Los Angeles', 'pop' : 0, 'nba' : 'Clippers'}
    ]
    nba_cities_df = pd.concat([nba_cities_df, pd.DataFrame(multiple_team_cities)])
    nba_cities = nba_cities_df.drop_duplicates("city").sort_values(by = "city")
    # Extract Series containing data: "Population by Region"
    population_by_region = nba_cities["pop"]
    # Data Cleaning: NBA Teams Dataset
    # Import Dataset
    nba_df = pd.read_csv("Assets/nba.csv")
    # Data Cleaning Operations
    nba_df = nba_df[nba_df.year == 2018].iloc[:, [0, 1, 2]]
    nba_df = nba_df[~nba_df.team.str.contains('Division')]
    nba_df.team = nba_df.team.replace('*', '')
    nba_df["city"] = 0
    for i in range(len(nba_cities_df)):
        nba_df["city"][nba_df.team.str.contains(nba_cities_df.nba.iloc[i])] = nba_cities_df.city.iloc[i]
        nba_df["team"][nba_df.team.str.contains(nba_cities_df.nba.iloc[i])] = nba_cities_df.nba.iloc[i]
    nba_df = nba_df.astype({'W': int, 'L': int})
    total_df = nba_df.groupby("city").sum()
    total_df["W/L"] = total_df["W"] / (total_df["W"] + total_df["L"])
    total_df = total_df.sort_values(by = "city")
    # Extract Series containing data: "W/L Ratio by Region"
    win_loss_by_region = total_df["W/L"]
    # Code Assessments
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"
    #raise NotImplementedError()
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [4]:
nba_correlation()

/Users/juandiegoocampo/.pyenv/versions/3.9.5/lib/python3.9/site-packages/pandas/core/series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-3-9c7220e1f3b1>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nba_cities_df['NBA'] = nba_cities_df['NBA'].str.strip()
<ipython-input-3-9c7220e1f3b1>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

-0.17657160252844614

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [132]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def mlb_correlation(): 
    # YOUR CODE HERE
    # Data Cleaning: US Cities Dataset
    # Import Dataset
    cities = pd.read_html("Assets/wikipedia_data.html")[1].iloc[:-1, [0, 3, 5, 6, 7, 8]]
    mlb_cities_df = cities.iloc[: ,[0, 1, -3]]
    mlb_cities_df['MLB'].replace('\[.*\]', '', inplace = True, regex = True)
    mlb_cities_df['MLB'].replace(['YankeesMets', 'DodgersAngels', 'GiantsAthletics', 'CubsWhite Sox', '—', ''], 
                                ['Yankees', 'Dodgers', 'Giants', 'Cubs', np.nan,np.nan], inplace = True)
    mlb_cities_df = mlb_cities_df.dropna()
    mlb_cities_df.columns = ["city", "pop", "mlb"]
    mlb_cities_df['pop'] = pd.to_numeric(mlb_cities_df['pop'])
    multiple_team_cities = [
        {'city' : 'New York City', 'pop' : 0, 'mlb' : 'Mets'},
        {'city' : 'Los Angeles', 'pop' : 0, 'mlb' : 'Angels'},
        {'city' : 'San Francisco Bay Area', 'pop' : 0, 'mlb' : 'Athletics'},
        {'city' : 'Chicago', 'pop' : 0, 'mlb' : 'White Sox'}
    ]
    mlb_cities_df = pd.concat([mlb_cities_df, pd.DataFrame(multiple_team_cities)])
    mlb_cities = mlb_cities_df.drop_duplicates("city").sort_values(by = "city")
    # Extract Series containing data: "Population by Region"
    population_by_region = mlb_cities["pop"]
    # Data Cleaning: MLB Teams Dataset
    # Import Dataset
    mlb_df = pd.read_csv("Assets/mlb.csv")
    # Data Cleaning Operations
    mlb_df = mlb_df[mlb_df.year == 2018].iloc[:, [0, 1, 2]]
    mlb_df = mlb_df[~mlb_df.team.str.contains('Division')]
    mlb_df.team = mlb_df.team.replace('*', '')
    mlb_df["city"] = 0
    for i in range(len(mlb_cities_df)):
        mlb_df["city"][mlb_df.team.str.contains(mlb_cities_df.mlb.iloc[i])] = mlb_cities_df.city.iloc[i]
        mlb_df["team"][mlb_df.team.str.contains(mlb_cities_df.mlb.iloc[i])] = mlb_cities_df.mlb.iloc[i]
    mlb_df = mlb_df.astype({'W': int, 'L': int})
    total_df = mlb_df.groupby("city").sum()
    total_df["W/L"] = total_df["W"] / (total_df["W"] + total_df["L"])
    total_df = total_df.sort_values(by = "city")
    # Extract Series containing data: "W/L Ratio by Region"
    win_loss_by_region = total_df["W/L"]
    # Code Assessments
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"
    #raise NotImplementedError()
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [133]:
mlb_correlation()

/Users/juandiegoocampo/.pyenv/versions/3.9.5/lib/python3.9/site-packages/pandas/core/series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-132-2711bd12b902>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mlb_df["city"][mlb_df.team.str.contains(mlb_cities_df.mlb.iloc[i])] = mlb_cities_df.city.iloc[i]
<ipython-input-132-2711bd12b902>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mlb_d

0.1505230448710485

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [276]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nfl_correlation(): 
    # YOUR CODE HERE
    # Data Cleaning: US Cities Dataset
    # Import Dataset
    cities = pd.read_html("Assets/wikipedia_data.html")[1].iloc[:-1, [0, 3, 5, 6, 7, 8]]
    nfl_cities_df = cities.iloc[: ,[0, 1, -4]]
    nfl_cities_df['NFL'].replace('\[.*\]', '', inplace = True, regex = True)
    nfl_cities_df['NFL'].replace(['GiantsJets', 'RamsChargers', '49ersRaiders', '—', '', '— '], 
                                ['Giants', 'Rams', '49ers', np.nan, np.nan, np.nan], inplace = True)
    nfl_cities_df = nfl_cities_df.dropna()
    nfl_cities_df.columns = ["city", "pop", "nfl"]
    nfl_cities_df['pop'] = pd.to_numeric(nfl_cities_df['pop'])
    multiple_team_cities = [
        {'city' : 'New York City', 'pop' : 0, 'nfl' : 'Jets'},
        {'city' : 'Los Angeles', 'pop' : 0, 'nfl' : 'Chargers'},
        {'city' : 'San Francisco Bay Area', 'pop' : 0, 'nfl' : 'Raiders'}
    ]
    nfl_cities_df = pd.concat([nfl_cities_df, pd.DataFrame(multiple_team_cities)])
    nlf_cities = nfl_cities_df.drop_duplicates("city").sort_values(by = "city")
    # Extract Series containing data: "Population by Region"
    population_by_region = nlf_cities["pop"]
    # Data Cleaning: NFL Teams Dataset
    # Import Dataset
    nfl_df = pd.read_csv("Assets/nfl.csv")
    # Data Cleaning Operations
    nfl_df = nfl_df[nfl_df.year == 2018].iloc[:, [-2, -4, 1]]
    nfl_df = nfl_df[~nfl_df.team.str.contains('FC')]
    nfl_df["city"] = 0
    for i in range(len(nfl_cities_df)):
        nfl_df["city"][nfl_df.team.str.contains(nfl_cities_df.nfl.iloc[i])] = nfl_cities_df.city.iloc[i]
        nfl_df["team"][nfl_df.team.str.contains(nfl_cities_df.nfl.iloc[i])] = nfl_cities_df.nfl.iloc[i]
    nfl_df = nfl_df.astype({'W': int, 'L': int})
    total_df = nfl_df.groupby("city").sum()
    total_df["W/L"] = total_df["W"] / (total_df["W"] + total_df["L"])
    total_df = total_df.sort_values(by = "city")
    # Extract Series containing data: "W/L Ratio by Region"
    win_loss_by_region = total_df["W/L"]
    # Code Assessments
    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"
    #raise NotImplementedError()
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [277]:
nfl_correlation()

/Users/juandiegoocampo/.pyenv/versions/3.9.5/lib/python3.9/site-packages/pandas/core/series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-276-afbde6fda750>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nfl_df["city"][nfl_df.team.str.contains(nfl_cities_df.nfl.iloc[i])] = nfl_cities_df.city.iloc[i]


0.004922112149349409

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [24]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nhl_correlation_data():
    # YOUR CODE HERE
    # Data Cleaning: US Cities Dataset
    # Import Dataset
    cities = pd.read_html("Assets/wikipedia_data.html")[1].iloc[:-1, [0, 3, 5, 6, 7, 8]]
    # Data Cleaning Operations
    nhl_cities_df = cities.iloc[: ,[0, 1, -1]]
    nhl_cities_df['NHL'].replace('\[.*\]', '', inplace = True, regex = True)
    nhl_cities_df['NHL'].replace(['RangersIslandersDevils', 'KingsDucks', '—', ''],['Rangers', 'Kings', np.nan, np.nan], inplace = True) 
    nhl_cities_df['NHL'] = nhl_cities_df['NHL'].str.strip()
    nhl_cities_df = nhl_cities_df.dropna()
    nhl_cities_df.columns = ["city", "pop", "nhl"]
    nhl_cities_df['pop'] = pd.to_numeric(nhl_cities_df['pop'])
    multiple_team_cities = [
        {'city' : 'New York City', 'pop' : 0, 'nhl' : 'Islanders'},
        {'city' : 'New York City', 'pop' : 0, 'nhl' : 'Devils'},
        {'city' : 'Los Angeles', 'pop' : 0, 'nhl' : 'Ducks'}
    ]
    nhl_cities_df = pd.concat([nhl_cities_df, pd.DataFrame(multiple_team_cities)])
    # Data Cleaning: NHL Teams Dataset
    # Import Dataset
    nhl_df = pd.read_csv("Assets/nhl.csv")
    # Data Cleaning Operations
    nhl_df = nhl_df[nhl_df.year == 2018].iloc[:, [0, 2, 3]]
    nhl_df = nhl_df[~nhl_df.team.str.contains('Division')]
    nhl_df.team = nhl_df.team.replace('*', '')
    nhl_df["city"] = 0
    for i in range(len(nhl_cities_df)):
        nhl_df["city"][nhl_df.team.str.contains(nhl_cities_df.nhl.iloc[i])] = nhl_cities_df.city.iloc[i]
        nhl_df["team"][nhl_df.team.str.contains(nhl_cities_df.nhl.iloc[i])] = nhl_cities_df.nhl.iloc[i]
    nhl_df = nhl_df.astype({'W': int, 'L': int})
    total_df = nhl_df.groupby("city").sum()
    total_df["W/L"] = total_df["W"] / (total_df["W"] + total_df["L"])
    return total_df

def nba_correlation_data():
    # YOUR CODE HERE
    # Data Cleaning: US Cities Dataset
    # Import Dataset
    cities = pd.read_html("Assets/wikipedia_data.html")[1].iloc[:-1, [0, 3, 5, 6, 7, 8]]
    nba_cities_df = cities.iloc[: ,[0, 1, -2]]
    nba_cities_df['NBA'].replace('\[.*\]', '', inplace = True, regex = True)
    nba_cities_df['NBA'].replace(['KnicksNets', 'LakersClippers', '—', ''],['Knicks', 'Lakers', np.nan, np.nan], inplace = True) 
    nba_cities_df['NBA'] = nba_cities_df['NBA'].str.strip()
    nba_cities_df = nba_cities_df.dropna()
    nba_cities_df.columns = ["city", "pop", "nba"]
    nba_cities_df['pop'] = pd.to_numeric(nba_cities_df['pop'])
    multiple_team_cities = [
        {'city' : 'New York City', 'pop' : 0, 'nba' : 'Nets'},
        {'city' : 'Los Angeles', 'pop' : 0, 'nba' : 'Clippers'}
    ]
    nba_cities_df = pd.concat([nba_cities_df, pd.DataFrame(multiple_team_cities)])
    # Data Cleaning: NBA Teams Dataset
    # Import Dataset
    nba_df = pd.read_csv("Assets/nba.csv")
    # Data Cleaning Operations
    nba_df = nba_df[nba_df.year == 2018].iloc[:, [0, 1, 2]]
    nba_df = nba_df[~nba_df.team.str.contains('Division')]
    nba_df.team = nba_df.team.replace('*', '')
    nba_df["city"] = 0
    for i in range(len(nba_cities_df)):
        nba_df["city"][nba_df.team.str.contains(nba_cities_df.nba.iloc[i])] = nba_cities_df.city.iloc[i]
        nba_df["team"][nba_df.team.str.contains(nba_cities_df.nba.iloc[i])] = nba_cities_df.nba.iloc[i]
    nba_df = nba_df.astype({'W': int, 'L': int})
    total_df = nba_df.groupby("city").sum()
    total_df["W/L"] = total_df["W"] / (total_df["W"] + total_df["L"])
    return total_df

def mlb_correlation_data(): 
    # YOUR CODE HERE
    # Data Cleaning: US Cities Dataset
    # Import Dataset
    cities = pd.read_html("Assets/wikipedia_data.html")[1].iloc[:-1, [0, 3, 5, 6, 7, 8]]
    mlb_cities_df = cities.iloc[: ,[0, 1, -3]]
    mlb_cities_df['MLB'].replace('\[.*\]', '', inplace = True, regex = True)
    mlb_cities_df['MLB'].replace(['YankeesMets', 'DodgersAngels', 'GiantsAthletics', 'CubsWhite Sox', '—', ''], 
                                ['Yankees', 'Dodgers', 'Giants', 'Cubs', np.nan,np.nan], inplace = True)
    mlb_cities_df = mlb_cities_df.dropna()
    mlb_cities_df.columns = ["city", "pop", "mlb"]
    mlb_cities_df['pop'] = pd.to_numeric(mlb_cities_df['pop'])
    multiple_team_cities = [
        {'city' : 'New York City', 'pop' : 0, 'mlb' : 'Mets'},
        {'city' : 'Los Angeles', 'pop' : 0, 'mlb' : 'Angels'},
        {'city' : 'San Francisco Bay Area', 'pop' : 0, 'mlb' : 'Athletics'},
        {'city' : 'Chicago', 'pop' : 0, 'mlb' : 'White Sox'}
    ]
    mlb_cities_df = pd.concat([mlb_cities_df, pd.DataFrame(multiple_team_cities)])
    # Data Cleaning: MLB Teams Dataset
    # Import Dataset
    mlb_df = pd.read_csv("Assets/mlb.csv")
    # Data Cleaning Operations
    mlb_df = mlb_df[mlb_df.year == 2018].iloc[:, [0, 1, 2]]
    mlb_df = mlb_df[~mlb_df.team.str.contains('Division')]
    mlb_df.team = mlb_df.team.replace('*', '')
    mlb_df["city"] = 0
    for i in range(len(mlb_cities_df)):
        mlb_df["city"][mlb_df.team.str.contains(mlb_cities_df.mlb.iloc[i])] = mlb_cities_df.city.iloc[i]
        mlb_df["team"][mlb_df.team.str.contains(mlb_cities_df.mlb.iloc[i])] = mlb_cities_df.mlb.iloc[i]
    mlb_df = mlb_df.astype({'W': int, 'L': int})
    total_df = mlb_df.groupby("city").sum()
    total_df["W/L"] = total_df["W"] / (total_df["W"] + total_df["L"])
    return total_df

def nfl_correlation_data(): 
    # YOUR CODE HERE
    # Data Cleaning: US Cities Dataset
    # Import Dataset
    cities = pd.read_html("Assets/wikipedia_data.html")[1].iloc[:-1, [0, 3, 5, 6, 7, 8]]
    nfl_cities_df = cities.iloc[: ,[0, 1, -4]]
    nfl_cities_df['NFL'].replace('\[.*\]', '', inplace = True, regex = True)
    nfl_cities_df['NFL'].replace(['GiantsJets', 'RamsChargers', '49ersRaiders', '—', '', '— '], 
                                ['Giants', 'Rams', '49ers', np.nan, np.nan, np.nan], inplace = True)
    nfl_cities_df = nfl_cities_df.dropna()
    nfl_cities_df.columns = ["city", "pop", "nfl"]
    nfl_cities_df['pop'] = pd.to_numeric(nfl_cities_df['pop'])
    multiple_team_cities = [
        {'city' : 'New York City', 'pop' : 0, 'nfl' : 'Jets'},
        {'city' : 'Los Angeles', 'pop' : 0, 'nfl' : 'Chargers'},
        {'city' : 'San Francisco Bay Area', 'pop' : 0, 'nfl' : 'Raiders'}
    ]
    nfl_cities_df = pd.concat([nfl_cities_df, pd.DataFrame(multiple_team_cities)])
    # Data Cleaning: NFL Teams Dataset
    # Import Dataset
    nfl_df = pd.read_csv("Assets/nfl.csv")
    # Data Cleaning Operations
    nfl_df = nfl_df[nfl_df.year == 2018].iloc[:, [-2, -4, 1]]
    nfl_df = nfl_df[~nfl_df.team.str.contains('FC')]
    nfl_df["city"] = 0
    for i in range(len(nfl_cities_df)):
        nfl_df["city"][nfl_df.team.str.contains(nfl_cities_df.nfl.iloc[i])] = nfl_cities_df.city.iloc[i]
        nfl_df["team"][nfl_df.team.str.contains(nfl_cities_df.nfl.iloc[i])] = nfl_cities_df.nfl.iloc[i]
    nfl_df = nfl_df.astype({'W': int, 'L': int})
    total_df = nfl_df.groupby("city").sum()
    total_df["W/L"] = total_df["W"] / (total_df["W"] + total_df["L"])
    return total_df

def sports_team_performance():
    # YOUR CODE HERE
    # Compute W/L Ratio per City 
    nfl_data = nfl_correlation_data()
    nba_data = nba_correlation_data()
    mlb_data = mlb_correlation_data()
    nhl_data = nhl_correlation_data()
    # Merging DataFrames
    nba_nfl = pd.merge(nba_data, nfl_data, on = 'city')
    nba_nhl = pd.merge(nba_data, nhl_data, on = 'city')
    mlb_nfl = pd.merge(mlb_data, nfl_data, on = 'city')
    mlb_nhl = pd.merge(mlb_data, nhl_data, on = 'city')
    mlb_nba = pd.merge(mlb_data, nba_data, on = 'city')
    nhl_nfl = pd.merge(nhl_data, nfl_data, on = 'city')
    # Compute P-Values
    pval_nba_nfl = stats.ttest_rel(nba_nfl['W/L_x'], nba_nfl['W/L_y'])[1]
    pval_nba_nhl = stats.ttest_rel(nba_nhl['W/L_x'], nba_nhl['W/L_y'])[1]
    pval_mlb_nfl = stats.ttest_rel(mlb_nfl['W/L_x'], mlb_nfl['W/L_y'])[1]
    pval_mlb_nhl = stats.ttest_rel(mlb_nhl['W/L_x'], mlb_nhl['W/L_y'])[1]
    pval_mlb_nba = stats.ttest_rel(mlb_nba['W/L_x'], mlb_nba['W/L_y'])[1]
    pval_nhl_nfl = stats.ttest_rel(nhl_nfl['W/L_x'], nhl_nfl['W/L_y'])[1]
    # Creating DataFrame
    # p_vals = [pval_nba_nfl, pval_nba_nhl, pval_mlb_nfl, pval_mlb_nhl, pval_mlb_nba, pval_nhl_nfl]
    p_values = {
        'NFL': {"NFL": np.nan, 'NBA': pval_nba_nfl, 'NHL': pval_nhl_nfl, 'MLB': pval_mlb_nfl},
        'NBA': {"NFL": pval_nba_nfl, 'NBA': np.nan, 'NHL': pval_nba_nhl, 'MLB': pval_mlb_nba},
        'NHL': {"NFL": pval_nhl_nfl, 'NBA': pval_nba_nhl, 'NHL': np.nan, 'MLB': pval_mlb_nhl},
        'MLB': {"NFL": pval_mlb_nfl, 'NBA': pval_mlb_nba, 'NHL': pval_mlb_nhl, 'MLB': np.nan}
    }
    p_values = pd.DataFrame(p_values)
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    # sports = ['NFL', 'NBA', 'NHL', 'MLB']
    # p_values = pd.DataFrame({k:p_vals for k in sports}, index=sports)
    # Code Assessments
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    #raise NotImplementedError()
    return p_values

In [25]:
sports_team_performance()

/Users/juandiegoocampo/.pyenv/versions/3.9.5/lib/python3.9/site-packages/pandas/core/series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-24-38d1cce4df98>:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nfl_df["city"][nfl_df.team.str.contains(nfl_cities_df.nfl.iloc[i])] = nfl_cities_df.city.iloc[i]
/Users/juandiegoocampo/.pyenv/versions/3.9.5/lib/python3.9/site-packages/pandas/core/series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030959,0.802384
NBA,0.941792,NaN,0.022316,0.951046
NHL,0.030959,0.022316,NaN,0.000712
MLB,0.802384,0.951046,0.000712,NaN
